## Compare two excel files

This Jupyter notebook compares two excel files and outputs a third excel file with the differences.

Restrictions:
* Each of the two excel file has only one excel sheet (tab). 
* The two excel files need to have the same column headers and the same number of columns.

In [ ]:
# Import the modules
import pandas as pd          # For the ease of reading and writing excel files and data manipulation

In [ ]:
# Parameters / Constants
## File names
df1_file        = "File1.xlsx"                         # Name of the first file to compare
df2_file        = "File2.xlsx"                         # Name of the second file to compare
output_file     = "Outputfile.xlsx"                    # Name of the output file for the result of the comparisation

## Key fields used in comparing the excel files (to give a row an unique index)
# Use a list of column headers to identify a row uniquely
indexfields     = ["Keyfield1", "Keyfield2", "Keyfield3"]

## Miscellaneous
NoValue         = "NoValue"                            # Which string value to use for NaN-values (Not-a-Number)
skip_rows       = 0                                    # Number of rows to skip in both excel files
dtype_object    = True                                 # True if indexfields should be treated as objects, False otherwise
unnamed_remove  = True                                 # True if unnamed columns need to be removed, False otherwise

## Status
status_column   = "status"                             # Name of the additional column for the status of each row
status_deleted  = "Deleted  (Present in 1, not in 2)"  # The row is present in dataframe 1, not in dataframe 2
status_new      = "New      (Not in 1, present in 2)"  # The row is not present in dataframe 1, but present in dataframe 2
status_same     = "Same     (1 == 2)"                  # The row is equal in dataframe 1 and dataframe 2
status_modified = "Modified (1 <> 2)"                  # The row is not equal in dataframe 1 and dataframe 2

## Dataframe
dataframe_id    = "Dataframe-ID"                       # Name of the additional column for the dataframe identifier
dataframe_1     = "df1"                                # Name of the dataframe identifier for dataframe 1
dataframe_2     = "df2"                                # Name of the dataframe identifier for dataframe 1
df1todf2marker  = " ---> "                             # Recognizable text to be used between 2 different values of the same cel

In [ ]:
# Follow-up depends on value of dtype_object
if dtype_object:
    # dtype_object is True
    dtype_info = dict.fromkeys(indexfields, object)
    # With this dictionary, all indexfields will be treated as Pandas Objects (no decimal representation of numeric key values)
else:
    # dtype_object is not True
    dtype_info = None
# Print the output of this step
print(dtype_info)

In [ ]:
# Read the excel files and place them in the data frames
df1 = pd.read_excel(df1_file, skiprows=skip_rows, dtype=dtype_info).fillna(NoValue)
print("File", df1_file, "has", df1.shape[0], "rows and", df1.shape[1], "columns")
df2 = pd.read_excel(df2_file, skiprows=skip_rows, dtype=dtype_info).fillna(NoValue)
print("File", df2_file, "has", df2.shape[0], "rows and", df2.shape[1], "columns")

In [ ]:
# Add the dataframe-ID to each dataframe
df1[dataframe_id] = dataframe_1
df2[dataframe_id] = dataframe_2

In [ ]:
# Show the first 5 lines of dataframe 1 and show the columns of dataframe 1
display(df1.head())
display(df1.columns)

In [ ]:
# Remove all "Unnamed: "-columns
if unnamed_remove:
    # Unnamed columns need to be removed
    columntitle = list(df1.columns)
    print("Old titles:", len(columntitle), "\n", columntitle)
    title_new = [item for item in columntitle if "Unnamed: " not in item]
    print("\nNew titles:", len(title_new), "\n", title_new)

    print("\nOld shapes:", df1.shape, df2.shape)
    df1 = df1[title_new].copy()
    df2 = df2[title_new].copy()
    print("New shapes:", df1.shape, df2.shape)
else:
    # Unnamed columns will not be removed
    print("No attempt in removing unnamed columns")

In [ ]:
# Set the index
df1 = df1.set_index(indexfields)
display(df1.head())

df2 = df2.set_index(indexfields)

In [ ]:
# Concatenate both dataframes
df3 = pd.concat([df1,df2],sort=False)
display(df3.head())

In [ ]:
# What are the shapes of the three dataframes
print("Shape of individual dataframes :", df1.shape, "and", df2.shape)
print("Shape of concatenated dataframe:", df3.shape)

In [ ]:
# Assign status to rows who only occur once
df3.loc[~df3.index.isin(df2.index), status_column] = status_deleted     # if not in df2 index then deleted
df3.loc[~df3.index.isin(df1.index), status_column] = status_new         # if not in df1 index then new

In [ ]:
# Are there rows who occor more than once?
df3.loc[~df3[status_column].isin([status_deleted, status_new])].head()

In [ ]:
# Collect rows of df2 who are also in df1
df2indf1 = df3.loc[(~df3[status_column].isin([status_deleted, status_new])) & (df3[dataframe_id] == dataframe_2)].copy()
print("df2indf1  :", df2indf1.shape)
# Remove rows in df3 who occur in df1 as well as df2, but who originate from df2:
# We want to keep only those rows who already have a assigned status or who originate from df1
print("df3 before:", df3.shape)
df3 = df3.loc[(df3[status_column].isin([status_deleted, status_new])) | (df3[dataframe_id] == dataframe_1)].copy()
print("df3 after :", df3.shape)

In [ ]:
# Show the first 5 rows
display(df3.head())

In [ ]:
# Functionality below this cell require that both dataframes have unique rows base on the index
assert len(df1.index.unique()) == df1.shape[0], "Index is not unique in df1"
assert len(df2.index.unique()) == df2.shape[0], "Index is not unique in df2"
# Now we have controlled that both dataframes have unique rows based on the index

In [ ]:
# Make a dictionary of the dataframe based on the index
dict_df2 = df2.to_dict("index")
print("Dictionary has", len(dict_df2),"entries.")

In [ ]:
def compare_row(row):
    # Follow-up depends on type of the field status_column
    if type(row[status_column]) == type(status_deleted):
        # Type of field status_column matches the type of an already assigned value, this record can be skipped.
        return row
    
    # Now the rows that didn't have an assigned value will be treated
    indexlist = list(row.index)       # list of field names of the row
    indexlist.remove(status_column)   # Remove the field name of the added status_column
    indexlist.remove(dataframe_id)    # Remove the field name of the added dataframe-identifier
    row2_dict = dict_df2[row.name]    # Get the row to compare from the dictionary (from dataframe 2)
    
    row[status_column] = status_same  # Assume that the rows are identical
    for element in indexlist:
        # Do the following for each field name 
        # Follow-up depends on the content of the same field in both rows
        if row[element] != row2_dict[element]:
            # Inhoud van het veld in beide rijen is niet gelijk
            row[status_column] = status_modified                                           # Markeer the row as change
            row[element] = str(row[element]) + df1todf2marker + str(row2_dict[element])    # Record the change
            print(row.name, element, row[element])                                         # Show the difference

    return row

In [ ]:
# Call the function above for each row
df4 = df3.apply(func=compare_row, axis=1)

In [ ]:
# Write the output file
df4.to_excel(output_file, merge_cells=False)
print("Output written to", output_file)